# Lesson06. Robot vision

教學主題： 處理來自機器人攝影機的畫面。

核心目標：即時影像與基礎處理。

前 30 分鐘 (理論)：介紹機器視覺與 OpenCV。

後 90 分鐘 (實作)：
* 即時圖傳： 編寫 Python，將 Go2 攝影機畫面即時顯示在筆電視窗中。

* 黑白電影濾鏡： 用 OpenCV 函式，將彩色畫面變成黑白畫面。

* 偵測邊緣： 嘗試 Canny 邊緣偵測，讓眼前世界變成素描畫。

* 私隱與倫理：討論人面識別應用下的隱私問題及原則。

* 責任與監督：討論人與機器人互動下的操作安全、社會相關法規及三層責任體系。

## 6.1 Import and initialize the client

In [ ]:
import time
import sys
from unitree_sdk2py.core.channel import ChannelSubscriber, ChannelFactoryInitialize
from unitree_sdk2py.go2.video.video_client import VideoClient
import cv2
import numpy as np

ChannelFactoryInitialize(0, "ens37")
video_client = VideoClient()  # Create a video client
video_client.SetTimeout(3.0)
video_client.Init()

## 6.2 Get and save image

In [3]:
print("##################GetImageSample###################")
code, data = video_client.GetImageSample()

if code != 0:
    print("get image sample error. code:", code)
else:
    imageName = "./go2_front_image.jpg"
    with open(imageName, "+wb") as f:
        f.write(bytes(data))
        
    print("ImageName:", imageName)


##################GetImageSample###################
ImageName: ./go2_front_image.jpg


## 6.3 View camera image as video

In [3]:
print("##################GetVideoSample###################")
code, data = video_client.GetImageSample()

# Request normal when code==0
while code == 0:
    # Get Image data from Go2 robot
    code, data = video_client.GetImageSample()

    # Convert to numpy image
    image_data = np.frombuffer(bytes(data), dtype=np.uint8)
    image = cv2.imdecode(image_data, cv2.IMREAD_COLOR)

    # Display image
    cv2.imshow("front_camera", image)
    # Press ESC to stop
    if cv2.waitKey(20) == 27:
        print("Stop video stream...")
        break

if code != 0:
    print("Get image sample error. code:", code)
else:
    # Capture an image
    imageName = "last_go2_front_image.jpg"
    cv2.imwrite(imageName, image)
    print("Save last image at:", imageName)

cv2.destroyWindow("front_camera")


##################GetVideoSample###################
Stop video stream...
Save last image at: last_go2_front_image.jpg


## 6.4 Canny 

In [4]:
import cv2
import numpy as np
# 假设前面已有video_client的初始化代码

print("##################GetImageSampleCanny###################")
code, data = video_client.GetImageSample()

if code != 0:
    print("get image sample error. code:", code)
else:
    # 保存原始彩色图像
    imageName = "./go2_front_image.jpg"
    with open(imageName, "+wb") as f:
        f.write(bytes(data))
    print("原始图像保存路径:", imageName)

    # 将图像数据转为字节数据转换为OpenCV可处理的格式
    image_data = np.frombuffer(bytes(data), dtype=np.uint8)
    color_image = cv2.imdecode(image_data, cv2.IMREAD_COLOR)

    if color_image is None:
        print("无法像解码失败")
    else:
        # 1. 转换为黑白图像（灰度图）
        gray_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)
        
        # 2. Canny边缘检测（参数可根据需要调整）
        # 第一个参数是低阈值，第二个是高阈值
        edges = cv2.Canny(gray_image, 50, 150)

        # 保存处理后的图像
        gray_image_name = "./go2_gray_image.jpg"
        edges_image_name = "./go2_canny_edges.jpg"
        cv2.imwrite(gray_image_name, gray_image)
        cv2.imwrite(edges_image_name, edges)
        print("黑白图像保存路径:", gray_image_name)
        print("边缘检测图像保存保存路径:", edges_image_name)

##################GetImageSampleCanny###################
原始图像保存路径: ./go2_front_image.jpg
黑白图像保存路径: ./go2_gray_image.jpg
边缘检测图像保存保存路径: ./go2_canny_edges.jpg
